## Load and import

In [18]:
# reload modules before running code
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import gc
import torch
torch.cuda.empty_cache()

gc.collect()

3774

In [20]:
!nvidia-smi

Tue Apr 15 09:55:38 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           On  |   00000000:00:04.0 Off |                    0 |
| N/A   62C    P0             38W /  250W |   15229MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [21]:
import sys
sys.path.insert(0, '..')
import config
import pandas as pd

In [ ]:
from src.functions import load_resources, get_current_date_string

from src.sum_fns import _read_file, load_fewshots, clean_process_bearblog, load_templates_zero_shot, summarize, batch_clean_summarization, batch_summarization

In [23]:
# worklog_summarization_prompt, desc_summarization_prompt = load_templates_zero_shot()
# worklog_summarization_prompt

## Read a single ticket 

In [24]:
ticket_number =  "TA0000017427771"   
# long tickets "TA0000017490033"  TA0000017415456 1
#TA0000017426456       TA0000017427771    "TA0000017488448"
file_path = f"../data/raw_summ_tickets/open_op_summ/{ticket_number}.rep"

ticket_content = _read_file(file_path)

In [25]:
# print(ticket_content)

## Cleaning and processing

In [ ]:
df_explode, len_logs = clean_process_bearblog(ticket_content, config.ticket_key, config.ticket_required_keys)

/home/jupyter/Ticket_summarization_resolution/DE_ticket_summarization_resolution/notebooks/../src/sum_fns.py:1298: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["processed_bearblog"] = df['processed_bearblog'].apply(extract_and_filter_text_within_symbols)
/home/jupyter/Ticket_summarization_resolution/DE_ticket_summarization_resolution/notebooks/../src/sum_fns.py:1300: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["processed_bearblog"] = df['day'] + ": " + df['processed_bearblog']


In [27]:
pd.options.display.max_colwidth = 0

In [28]:
df_explode[['BearbLog', 'cleaned_bearblog', 'Benutzer', 'processed_bearblog']]
# view all colwidth

,BearbLog,cleaned_bearblog,Benutzer,processed_bearblog
0,"neuer Kundenkontakt angelegt:\n initiiert durch: Vodafone\n Grund: Statusmeldung\n Kontaktname: Kleinitzke\n CC-T: 0049\n Tel.: 05273/905208\n erreicht: erreicht\n Funktionsbest. d. Kunde: \n KG geführt: \n Kurzbeschr.(intern): s\n Kein Termin: \n Technikerbesuch: \n Kundeninfo: Ansprechpartner konnte wegen statusmeldung erreicht werden. \n angerufen unter 0171/9942259, Basic checks werden gemacht\n \n","[[Kunde kontaktiert, Ansprechpartner konnte wegen statusmeldung erreicht werden\nangerufen unter 0171/9942259, Basic checks werden gemacht]]",ammar.fahmy1,"06-01-2025: Kunde kontaktiert, Ansprechpartner konnte wegen statusmeldung erreicht werden\nangerufen unter 0171/9942259, Basic checks werden gemacht"
0,"Neuer Auftrag 'AT0000010923012'angelegt. Problembeschreibung: Kunde: 2710014138; \n Vodafone GmbH\n Referenznummer Supportfunktion\n (Support-Auftragsnummer)\n 4447628550229\n \n Funktion\n LINERESET\n Vertragsnummer\n 3051309947\n Ausführung erfolgreich\n ja\n Meldung\n Supportfunktion wurde erfolgreich durchgeführt\n Zeitpunkt der Ausführung\n 06.01.2025, 12:14\n \n \n",[Neuer Auftrag 'AT0000010923012'angelegt. Problembeschreibung: Kunde: 2710014138;\nVodafone GmbH\nReferenznummer Supportfunktion\n(Support-Auftragsnummer)\nFunktion\nLINERESET\nVertragsnummer\nAusfuhrung erfolgreich\nja\nMeldung\nSupportfunktion wurde erfolgreich durchgefuhrt\nZeitpunkt der Ausfuhrung],ammar.fahmy1,06-01-2025: Ein neuer auftrag wurde angelegt zugewiesen Deutsche Telekom. Problembeschreibung: Kunde: 2710014138;\nVodafone GmbH\nReferenznummer Supportfunktion\n(Support-Auftragsnummer)\nFunktion\nLINERESET\nVertragsnummer\nAusfuhrung erfolgreich\nja\nMeldung\nSupportfunktion wurde erfolgreich durchgefuhrt\nZeitpunkt der Ausfuhrung
0,"neuen Eskalationskontakt $EskaID$ angelegt:\n Medium: Mail\n initiiert durch: Vodafone\n Grund: Eskalationsstart\n Betreff: #ED-Level 1#\n Kurzbeschr. (intern): #EG-Leitungskündigung#\n \n Frage: Wurde Ihre Leitung ohne Ihre Zustimmung oder Absprache gekündigt?\n Die Leitungskündigung erfolgte ohne Zustimmung des Kunden\n \n ######ED######\n Eskalation durch: Matthias Gottlob, VB \n Eskalation Zeit: 13:50Uhr \n Eskalation Mail an: G-FMEP-TSSE2\n Rückruf durch: Fachabteilung\n Rückruf um: bei neuem Status\n ######ED######\n \n","[Eine neue eskalation wurde initiert, da , [Wurde Ihre Leitung ohne Ihre Zustimmung oder Absprache gekündigt?, Die Leitungskündigung erfolgte ohne Zustimmung des Kunden\n######ED######\nEskalation durch: Matthias Gottlob, VB\nEskalation Zeit: 13:50Uhr\nEskalation Mail an: G-FMEP-TSSE2\nRückruf durch: Fachabteilung\nRückruf um: bei neuem Status\n######ED######]]",luzie.jozwiak01,"06-01-2025: Eine neue eskalation wurde initiert, da . Wurde Ihre Leitung ohne Ihre Zustimmung oder Absprache gekündigt?. Die Leitungskündigung erfolgte ohne Zustimmung des Kunden\n######ED######\nEskalation durch: Matthias Gottlob, VB\nEskalation Zeit: 13:50Uhr\nEskalation Mail an: G-FMEP-TSSE2\nRückruf durch: Fachabteilung\nRückruf um: bei neuem Status\n######ED######"
0,"neuer Kundenkontakt angelegt:\n initiiert durch: Vodafone\n Grund: Eskalationsmeldung\n Kontaktname: Matthias Gottlob, VB \n CC-T: \n Tel.: \n erreicht: erreicht\n Funktionsbest. d. Kunde: \n KG geführt: \n Kurzbeschr.(intern): #EG-Leitungskündigung#\n\n\n \n Frage: Wurde Ihre Leitung ohne Ihre Zustimmung oder Absprache gekündigt?\n Die Leitungskündigung erfolgte ohne Zustimmung des Kunden\n \n #ED Eskalation Start#\n #ED Level 1#\n ######ED######\n Eskalation durch: Matthias Gottlob, VB \n Eskalation Zeit: 13:50Uhr \n Eskalation Mail an: G-FMEP-TSSE2\n Rückruf durch: Fachabteilung\n Rückruf um: bei neuem Status\n ######ED######\n \n Beim Kunden wurde ohne Absprache einen Umschaltung BIC auf MPLS durchgeführt. Es gab \n hierzu keine Abstimmung mit dem Kunden! Jetzt funktioniert die Telefonie nicht, da der \n SIP nicht umgezogen wurde. Das Problem gab es schonmal bei einer Umschaltung, damal

In [29]:
df_explode['Benutzer'].value_counts()

Benutzer
ammar.fahmy1         5
luzie.jozwiak01      4
yehia.elseify        2
mohamed.abuhafez1    1
said.muhammed        1
Name: count, dtype: int64

## Inspect the cleaned segments

In [30]:
for v in df_explode.processed_bearblog.values:
    print(
        v,
        "\n",
        "---------------------------------------------------------------------------------\n",
    )

06-01-2025: Kunde kontaktiert, Ansprechpartner konnte wegen statusmeldung erreicht werden
angerufen unter 0171/9942259, Basic checks werden gemacht 
 ---------------------------------------------------------------------------------

06-01-2025: Ein neuer auftrag wurde angelegt zugewiesen Deutsche Telekom. Problembeschreibung: Kunde: 2710014138;
Vodafone GmbH
Referenznummer Supportfunktion
(Support-Auftragsnummer)
Funktion
LINERESET
Vertragsnummer
Ausfuhrung erfolgreich
ja
Meldung
Supportfunktion wurde erfolgreich durchgefuhrt
Zeitpunkt der Ausfuhrung 
 ---------------------------------------------------------------------------------

06-01-2025: Eine neue eskalation wurde initiert, da . Wurde Ihre Leitung ohne Ihre Zustimmung oder Absprache gekündigt?. Die Leitungskündigung erfolgte ohne Zustimmung des Kunden
######ED######
Eskalation durch: Matthias Gottlob, VB
Eskalation Zeit: 13:50Uhr
Eskalation Mail an: G-FMEP-TSSE2
Rückruf durch: Fachabteilung
Rückruf um: bei neuem Status
######ED

In [31]:
model = load_resources("Qwen2.5-32B-Ins_Q2_K_L")

In [32]:
model

ChatOllama(model='hf.co/bartowski/Qwen2.5-32B-Instruct-GGUF:Q2_K_L', temperature=0.0)

## Summarize the ticket

In [33]:
%%time
results = summarize(
    df_explode,
    model,
    # worklog_summarization_prompt,
    # desc_summarization_prompt,
)

CPU times: user 12.8 ms, sys: 4.5 ms, total: 17.3 ms
Wall time: 1min 7s


In [34]:
print(results)

Problembeschreibung: Keine Telefonie möglich nach MPLS-Migration am Standort Halle. Störung Internet - Verbindungsabbrüche.
06-01-2025: Kunde kontaktiert, Ansprechpartner konnte wegen statusmeldung erreicht werden. Ein neuer Auftrag wurde angelegt und zugewiesen an Deutsche Telekom mit der Beschreibung des Problems durch den Kunden. Die Ausführung war erfolgreich und die Supportfunktion LINERESET wurde durchgeführt.  
06-01-2025: Eskalation wegen Leitungskündigung ohne Zustimmung des Kunden initiiert durch Matthias Gottlob, VB. Es gab keine Abstimmung zur Umschaltung BIC auf MPLS und die SIP war nicht umgezogen. Der Router ist seit 2025-01-02 ausgefallen.  
06-01-2025: Kunde kontaktiert, Asp. Andree Kleinitzke erreicht und über Eskalation informiert. Die alte Leitung war weggefallen und die neue Leitung lag vor Ort.  
06-01-2025: Auftrag geschlossen von Deutsche Telekom mit Lösungsbeschreibung: Fehler ICP, Leitung gekündigt seit 30.12.2024.  
07-01-2025: Router über Regelweg erreichbar

In [35]:
# Ticket 771 

# •	Die Basic Checks wurden mit dem Kunden durchgefuehrt. Die Deutsche Telekom Netzkontrolle wurde beauftragt.
# •	Der Kunde wurde informiert, dass seine Eskalation aufgenommen ist. Die Änderung auf MPLS Leitung ist ebenfalls erfolgt aber ohne den Schwenk der Telefonie. 
# •	Die Deutsche Telekom meldete, dass die Leitung gekuendigt ist.
# •	Der Kunde hat die alte Leitung geschaltet und die eine neue abonniert, alles funktioniert gut mit der neuen Leitung außer VoIP.
# •	Der Kunde ist nicht erreichbar, weder per Mail noch telefonisch.


# • The basic checks were carried out with the customer. Deutsche Telekom network control was commissioned.
# • The customer has been informed that their escalation has been processed. The change to MPLS line was also made but without the telephony change. 
# • Deutsche Telekom reported that the line was terminated.
# • The customer switched the old line and subscribed to the new one, everything works fine with the new line except VoIP.
# • The customer cannot be reached, neither by email nor by telephone.

In [36]:
# long ticket 

# •	Paketverluste auf der Verbindung wurde durch Pingtest festgestellt.
# •	Ein Vodafone Techniker wurde beauftragt, um das Modem zu tauschen.
# •	Nach dem Modemtausch können wir weder Paketverluste noch Abbrüche feststellen.
# •	Der Kunde wurde informiert, dass wir somit das Ticket schließen werden.
# •	Ein neuer Alarm, dass die Störung wieder aufgetreten ist.
# •	Weitere Paketverluste wurden festgestellt und die Deutsche Telekom wurde beauftragt.
# •	Die Deutsche Telekom meldete zurück, dass kein Fehler feststellbar ist.
# •	Die Deutsche Telekom wurde wieder beauftragt und nochmal kein Fehler festgestellt.
# •	Die Deutsche Telekom wurde zum 3. Mal beauftragt. Der Fall wurde bei DTAG eskaliert.
# •	Der Fall ist weiterhin bei der Telekom wegen Langzeitmessung

# • Packet loss on the connection was detected by ping test.
# • A Vodafone technician was hired to replace the modem.
# • After replacing the modem, we cannot detect any packet losses or terminations.
# • The customer has been informed that we will therefore close the ticket.
# • A new alarm that the fault has occurred again.
# • Further packet losses were discovered and Deutsche Telekom was commissioned.
# • Deutsche Telekom reported back that no errors could be detected.
# • Deutsche Telekom was commissioned again and again no errors were found.
# • Deutsche Telekom was commissioned for the third time. The case was escalated to DTAG.
# • The case is still with Telekom because of long-term measurement


In [37]:
# Das Modem ist offline. Kabel Deutschland wurde beauftragt. 

# Kabel Deutschland meldete zurück, dass alles von ihrer Seite in Ordnung ist. 

# Das Modem ist weiterhin offline. Modem kann nicht resettet werden. Beim Aufruf der Schnittstelle zum Resetten des Modems ist ein Fehler aufgetreten. 

# Der Kunde meldete, dass die Störung weiterhin vorhanden ist. 

# Kabel Deutschland wurde wieder mit der Entstörung beauftragt. 


# The modem is offline. Kabel Deutschland was commissioned. 

# Kabel Deutschland reported back that everything was fine on their part. 

# The modem is still offline. Modem cannot be reset. An error occurred when calling the interface to reset the modem. 

# The customer reported that the problem still exists. 

# Kabel Deutschland was again commissioned with the troubleshooting.


## Translating results for readability

In [38]:
trans_prompt = f"translate from German to English: {results}"

In [39]:
%%time
print(model.invoke(trans_prompt).content)

Problem description: No telephony possible after MPLS migration at the Halle site. Internet disruption - connection failures.
January 6, 2025: Customer contacted; point of contact could be reached due to status message. A new order was created and assigned to Deutsche Telekom with the problem described by the customer. The execution was successful, and the support function LINERESET was performed.
January 6, 2025: Escalation initiated by Matthias Gottlob, VB, regarding unauthorized line cancellation without customer consent. There was no coordination for switching BIC to MPLS, and SIP was not migrated. The router has been down since January 2, 2025.
January 6, 2025: Customer contacted; Asp. Andree Kleinitzke reached and informed about the escalation. The old line had fallen away, and the new line was on-site.
January 6, 2025: Order closed by Deutsche Telekom with solution description: ICP error, line canceled since December 30, 2024.
January 7, 2025: Router accessible via regular route

In [40]:
asd # to force the code to stop 

NameError: name 'asd' is not defined

## Batch clean tickets

In [ ]:
with open("../data/cleaned_summ_tickets/ticket_summaries.txt", 'w', encoding='utf-8') as f:
    results_all = batch_clean_summarization()
    f.writelines(results_all)

## Batch clean and summarize tickets

In [41]:
%%time 

current_date = get_current_date_string()

with open(f"../data/evaluation/summarization/{current_date}.txt", 'w', encoding='utf-8') as f:
    results_all = batch_summarization(model = model)
    f.writelines(results_all)

TA0000017497770.rep


/home/jupyter/Ticket_summarization_resolution/DE_ticket_summarization_resolution/notebooks/../src/sum_fns.py:1298: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["processed_bearblog"] = df['processed_bearblog'].apply(extract_and_filter_text_within_symbols)
/home/jupyter/Ticket_summarization_resolution/DE_ticket_summarization_resolution/notebooks/../src/sum_fns.py:1300: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["processed_bearblog"] = df['day'] + ": " + df['processed_bearblog']


TA0000017488448.rep


/home/jupyter/Ticket_summarization_resolution/DE_ticket_summarization_resolution/notebooks/../src/sum_fns.py:1298: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["processed_bearblog"] = df['processed_bearblog'].apply(extract_and_filter_text_within_symbols)
/home/jupyter/Ticket_summarization_resolution/DE_ticket_summarization_resolution/notebooks/../src/sum_fns.py:1300: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["processed_bearblog"] = df['day'] + ": " + df['processed_bearblog']


TA0000017427771.rep


...

TA0000017415456 1.rep


/home/jupyter/Ticket_summarization_resolution/DE_ticket_summarization_resolution/notebooks/../src/sum_fns.py:1298: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["processed_bearblog"] = df['processed_bearblog'].apply(extract_and_filter_text_within_symbols)
/home/jupyter/Ticket_summarization_resolution/DE_ticket_summarization_resolution/notebooks/../src/sum_fns.py:1300: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["processed_bearblog"] = df['day'] + ": " + df['processed_bearblog']


TA0000017444190 - Summarization.rep


/home/jupyter/Ticket_summarization_resolution/DE_ticket_summarization_resolution/notebooks/../src/sum_fns.py:1298: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["processed_bearblog"] = df['processed_bearblog'].apply(extract_and_filter_text_within_symbols)
/home/jupyter/Ticket_summarization_resolution/DE_ticket_summarization_resolution/notebooks/../src/sum_fns.py:1300: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["processed_bearblog"] = df['day'] + ": " + df['processed_bearblog']


CPU times: user 8.3 s, sys: 20.8 ms, total: 8.33 s
Wall time: 14min 14s


## Summarization without few shots for the TSLAM model 

In [ ]:
# from transformers import AutoTokenizer

# def count_tokens(text, model_path = "NetoAISolutions/TSLAM-4B"):
#     """
#     Count the number of tokens in a given text using the tokenizer from the specified model.

#     Args:
#         text (str): The input text to tokenize.
#         model_path (str): The path or name of the model to load the tokenizer from.

#     Returns:
#         int: The number of tokens in the text.
#     """
#     # Load the tokenizer
#     tokenizer = AutoTokenizer.from_pretrained(model_path)
    
#     # Tokenize the text and count the tokens
#     tokens = tokenizer.encode(text, return_tensors="pt")
#     return tokens.shape[1]

In [ ]:
# # Count tokens in a text
#     worklog,
#     desc,
#     produkt,
#     model,
#     worklog_summarization_prompt,
#     desc_summarization_prompt,
text = desc_summarization_prompt.template
# text = worklog_summarization_prompt.template 
# text = worklog
token_count = count_tokens(text)
print(f"Number of tokens: {token_count}")

In [ ]:
# prompt = """ Given the following Ticket worklog numbered by date of sent message, summarize the worklog in clear bullet points showing what happened in the ticket sequentially
# Your answer must be in german language

# worklog:
# 06.09.2024 09:36:47: Neue Kundeninfo de Kundeninfo zur Kenntnis genommen
# 06.09.2024 09:57:12: neuer Kundenkontakt initiiert durch Vodafone mit Kevin Meyer aufgrund einer Statusmeldung Abbrueche Telekom beauftragt Herr Bittner 0152 06018273 erreicht und informiert Abbrueche Telekom beauftragt Herr Bittner 0152 06018273 erreicht und informiert
# 06.09.2024 09:58:41: Neuer Auftrag 'AT0000010855943'angelegt Problembeschreibung  #C4A# 0152 06018273 #Sync-Abbrüche [Öffnungszeiten & Ansprechpartner]  Herr Bittner 0152 06018273 Starke Abbrueche sind vorhanden und die Messung zeigt sich, dass es Alarme bei der Leitung gibt Die Leitung wurde neugestartet und es gibt keine Verbesserung Bitte die Leitung prüfen und ggf entstören Danke,
# 06.09.2024 13:26:39: Auftrag AT0000010855943 geschlossen Lösungsbeschreibung  TCom Entstörmeldung von  Carrier Telekom IP-Bitstrom Eingang  06 09 2024 13 26 35 Störungsbeseitigung  06 09 2024 13 18 00 Lösung d Beh   Nein Abschlussmeldung  Kein Fehler festgestellt (Techniker hat beim Endkunden geprüft) Meldung  7005   Kein Fehler festgestellt (Techniker hat beim Endkunden geprüft) ; Bemerkung  [IHC]0A;
# 06.09.2024 13:40:26: Neuer Auftrag 'AT0000010856212'angelegt Problembeschreibung  Kein Fixtermin -Eintrag [Was meldet der Kunde?]  Kunde bemängelt Abbrüche [Was sehen wir (Störungsbeschreibung)]  Abbrüche sind ersichtlich [Wie wurde bisher entstört? (Auftragshistorie)]  Basic checks wurden mit dem Kunden durchgefuehrt und es gibt keine Verbesserung Verkabelung, Stromversorgung und Reset des Equipments wurden durchgefuehrt Ein Telekom-Techniker war vor Ort und hat keinen Fehler festgestellt [Was soll wo vor Ort gemacht werden?]  Bitte die Verkabelung und die Geräte beim Kunden vor Ort prüfen und ggf tauschen Modemstyp  ZyXEL VMG1312 Falls der Fehler danach weiterhin besteht, dann soll eine Messung mit Argus auf der TAE Dose durchgeführt werden [Öffnungszeiten & Ansprechpartner]  Name & RN des ASP, Öffnungszeiten Herr Bittner 0152 06018273 [VF-Fachteam {Teamhotline}]  0211/5334466 Danken und Gruß
# 09.09.2024 09:54:07: neuer Kundenkontakt initiiert durch Vodafone mit GFTD_FS aufgrund einer Statusmeldung Neuer Anhang vom TTWOS zum Auftrag AT0000010856212
# 09.09.2024 10:02:24: Lösungskoordinator  ENTERPRISE24-IM-ZV Lösungsbeschreibung  Das Modem war defekt
# 09.09.2024 10:10:37: Auftrag AT0000010856212 geschlossen Lösungsbeschreibung  Rückmeldeüberschrift  Behoben - HW - Tausch Rückmeldebeschreibung  09 09 2024 10 07 43 GFTD_Service_Desk GFTD_FS Das Modem ZyXEL VMG1312 war Defekt und wurde inkl Netzteil und TAE/DSL Kabel ausgetauscht Der Cisco Router soeepw8003 ist wieder über den Regelweg erreichbar Das Ticket kann nach Absprache geschlossen werden Techniker On Site  Yes DNOC-Call  Yes DNOC-Name  Ticketersteller Naview-Done  Yes Datum für die Lösung  09 09 2024 10 07 00
# 09.09.2024 10:12:37: neuer Kundenkontakt initiiert durch Vodafone mit Kevin Meyer aufgrund einer Abschlussmeldung Modem war defekt Router laeuft wieder ueber den Regelweg Herr Meyer 0251/2021161 erreicht Er testet und meldet sich im Ticket zurueck Modem war defekt Router laeuft wieder ueber den Regelweg Herr Meyer 0251/2021161 erreicht Er testet und meldet sich im Ticket zurueck
# 09.09.2024 13:11:35: neuer Kundenkontakt initiiert durch Vodafone mit Kevin Meyer aufgrund einer Abschlussmeldung Ihre Störung wurde behoben
# 09.09.2024 13:16:09: Lösungskoordinator  Lösungsbeschreibung
# 09.09.2024 14:37:33: neuer Kundenkontakt initiiert durch Vodafone mit Kevin Meyer aufgrund einer Abschlussmeldung Enterprise Abschlussmeldung final (RFO) Ihre Störung wurde behoben
# 09.09.2024 14:38:08: Lösungskoordinator  FMEP TSSE2 Lösungsbeschreibung  Störungsgrund  GK-Produkte | Vodafone CPE | Defekt, Tausch -> priv HW Kunde | ZyXel-Modem - GKPFC#229 006 gesetzt am   09 09 2024 14 38 06


# German summary:

# """

# print(mis_model.invoke(prompt).content)


# def starter(data):
#     ticket,Problembeschr,desc,Technik,final_wroklog,produkt,dates,users= step1_processing(data)
#     Technik = ['Daten IP']* len(desc)
#     data_dict = {
#         'Ticketnr': ticket,
#         'Problembeschr': Problembeschr,
#         'exakte_Problem': desc,
#         'Dienst/Technik': Technik,
#         'Datum': dates,
#         'Benutzer': users,
#         'BearbLog': final_wroklog,
#         'Dienst/Produkt':produkt,
#     }
#     df = pd.DataFrame(data_dict)
#     df_explode =df.explode(['Datum', 'Benutzer', 'BearbLog'])
#     df_explode.BearbLog = df_explode.BearbLog.apply(lambda x: x.replace('\r', '').replace('\\r', '').replace('\\n','\n').replace('\\t\\t','\n').replace("\t\t\t","\n"))
#     df_explode.exakte_Problem = df_explode.exakte_Problem.apply(lambda x: x.replace('\r', '').replace('\\r', '').replace('\\n','\n').replace('\\t\\t','\n').replace("\t\t\t","\n"))
#     df_explode=df_explode.sort_values(by=['Ticketnr','Datum'])
#     df_explode['processed_bearblog'] =df_explode.BearbLog.apply(lambda x:clean_text(extract_fields_with_keys(patterns, x)))
#     df_explode['processed_bearblog'] = df_explode['processed_bearblog'].apply(filter_logs)
#     df_explode['new_processed_bearblog'] = df_explode.processed_bearblog.apply(lambda x: clean_text2(x))
#     df_explode['Dienst/Produkt'] =df_explode['Dienst/Produkt'].apply(lambda x: x.replace(':',''))
#     df_explode['Benutzer'] =df_explode['Benutzer'].apply(lambda x: x.replace('\n',''))
#     return df_explode

## Cleaning trial 

### clean whitespace 

In [ ]:
from src.sum_fns import clean_whitespace
import re 

text = """
'Router logs beweisen, dass der Router in Ordnung\n================================================'
"""

filtered_lines = [line for line in re.split(r"\. ?\n", text) ]

filtered_lines
# clean_whitespace(text)


### sentence split



In [ ]:
import re
text = """Eine neue eskalation wurde initiert, da , [Frage: Wurde Ihre Leitung ohne Ihre Zustimmung oder Absprache gekündigt?, Die Leitungskündigung erfolgte ohne Zustimmung des Kunden, G-FMEP-TSSE2\nRückruf durch: Fachabteilung\nRückruf um: bei neuem Status\n######ED######"""

re.sub(r"#+.*?#+", "", text)


In [ ]:
text = """Router ueber Regelweg erreichbar"""

doc = nlp(text)
for i in doc.sents:
    print(i)

In [ ]:
from src.sum_fns import split_into_sentences

# split_into_sentences(text)



In [ ]:

def split_into_sentences(paragraph):
    """
    Split a paragraph into sentences using spaCy.
    
    Parameters:
        paragraph (str): The paragraph to split.
        
    Returns:
        list: List of sentences.
    """
    doc = nlp(paragraph) if paragraph else nlp("")
    return [sent.text.strip() for sent in doc.sents]

### has verb/adj

In [ ]:
import spacy
nlp = spacy.load("de_core_news_md")

In [ ]:
text = "Router logs beweisen, dass der Router in Ordnung"
doc = nlp(text)
[token.pos_ for token in doc]

# has_verb_or_adj = any(token.pos_ in {"VERB", "ADJ", "AUX"} for token in doc) # AUX is auxiliary verb


### filter DOCSIS

In [ ]:
import re

def filter_text_DOCSIS(text):
    """
    Filters text, removing lines matching patterns and subsequent lines until a single newline.

    Args:
        input_text: The input text as a string.

    Returns:
        The filtered text as a string.
    """
    lines = text.splitlines()
    filtered_lines = []
    skip_mode = False

    for line in lines:
        if skip_mode:
            if not line.strip():
                skip_mode = False
            else:
                continue
        if "DOCSIS-Status" in line or re.match(r"\s{5,}-", line):
            skip_mode = True
            continue
        filtered_lines.append(line)

    return "\n".join(filtered_lines)

text = """
      
   06.01.2025 13:51:06 mohab.khafagy1
   Modem weiterhin Offline
   
   DOCSIS-Status        -
   Ping (min/max/avg)        -
   Verbindungsstatus        offline 2 Tage, 01:01:36
   Partial Service Mode Status        -
   Status Pegelwerte        Unbekannt
   
   #############################


   """
filter_text_DOCSIS(text)

### remove logs terminal

In [ ]:
from src.sum_fns import remove_logs_terminal_tbls_from_paragraphs
text = """
##########################\n Modem offline;\n Modem\n Voller Name -\n Provider -\n CM MAC-Adresse a8:70:5d:1b:74:de \n MTA MAC-Adresse -\n IP-Adresse -\n Firmware-Version -\n Firmware-Datei -\n Letztes Firmware-Update 29.11.2024, 10:16:01\n Mit Strom versorgt seit -\n Letzte Registrierung an der CMTS 1 Woche, 1 Tag, 15:04:41\n DOCSIS-Status -\n Ping (min/max/avg) -\n Verbindungsstatus offline 02:21:29\n Partial Service Mode Status -\n Status Pegelwerte Unbekannt\n \n
"""
remove_logs_terminal_tbls_from_paragraphs(text)
# I am still facing a problem here with human readable function

### is terminal?

In [ ]:
text = "    SHLEDEKAHS8004#ping vrf edekahs 010.038.174.001 so vl1 repeat 100"
from src.sum_fns import is_terminal_command
is_terminal_command(text)

### remove table lines

In [ ]:
from src.sum_fns import remove_table_lines
text = "Der RTR ist lt. XMON Analyse seit dem  2025-01-02 15:21Uhr ausgefallen"
remove_table_lines(text)

### failed trial

In [ ]:
nlp = spacy.load("de_core_news_sm") #You can use a german model as well, but I will show an english example.
nlp.add_pipe('sentencizer', config={"punct_chars": [".\n", "!", "?", "-\n"]}, before="parser")

import re 

def split_text(text):
  """Splits text based on the provided punctuation pattern."""
  punct_chars = [r"\.\n", r"!", r"\?", r"-\n"]
  pattern = "(" + "|".join(punct_chars) + ")"
  return [item.strip() for item in re.split(pattern, text) if item.strip()]

# "   -"
text = """
DOCSIS-Status         -
Ping (min/max/avg)        -
Verbindungsstatus        offline 2 Tage, 01:01:36
Partial Service Mode Status        -
Status Pegelwerte        Unbekannt
"""
# text = """
# DOCSIS-Status -\n Ping (min/max/avg) -\n Verbindungsstatus offline 02:21:29\n Partial Service Mode Status -\n Status Pegelwerte Unbekannt\n \n
# """
# split_text(text, pattern)
# foo = [item for item in text.split("-\n") if item.strip() != ""]
# print(foo)

doc = nlp(text)

for sent in doc.sents:
    print("--------")
    print(sent.text.strip())

    
# text = """
# Modem weiterhin Offline\n \n DOCSIS-Status -\n 
# Ping (min/max/avg) -\n Verbindungsstatus offline 2 Tage, 01:01:36\n Partial Service Mode Status -\n Status Pegelwerte Unbekannt\n \n #############################\n \n Modem kann nicht resettet werden\n \n Beim Aufruf der Schnittstelle zum Resetten des Kabelmodems a8:70:5d:1b:74:de ist ein \n Fehler aufgetreten.\n \n
# """

## End of cleaning trial